# Experiment 115: Proper GNN Implementation

**Goal**: Implement a GNN that operates on molecular graphs to potentially change the CV-LB relationship.

**Key Requirements**:
1. Use the SAME model class in CV computation AND submission cells
2. Use PyTorch Geometric with GCNConv
3. Use Morgan fingerprints as node features
4. Ensure model class is `GNNModel` everywhere

**Hypothesis**: GNN may capture structural information that tabular models miss, potentially changing the CV-LB relationship.

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

In [ ]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

In [ ]:
# Load SMILES lookup for Morgan fingerprints
SMILES_DF = pd.read_csv(f'{DATA_PATH}/smiles_lookup.csv', index_col=0)
print(f'SMILES lookup: {SMILES_DF.shape}')
print(SMILES_DF.head())

In [ ]:
# Generate Morgan fingerprints for all solvents
from rdkit import Chem
from rdkit.Chem import AllChem

def get_morgan_fingerprint(smiles, radius=2, n_bits=1024):
    """Generate Morgan fingerprint from SMILES"""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(n_bits)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    return np.array(fp)

# Create Morgan fingerprint lookup
MORGAN_FP = {}
for solvent in SMILES_DF.index:
    smiles = SMILES_DF.loc[solvent, 'solvent smiles']  # Fixed column name
    if isinstance(smiles, str):
        MORGAN_FP[solvent] = get_morgan_fingerprint(smiles)
    else:
        MORGAN_FP[solvent] = np.zeros(1024)

print(f'Generated Morgan fingerprints for {len(MORGAN_FP)} solvents')

In [ ]:
# Featurizer that combines Spange + DRFP + Morgan fingerprints
class CombinedFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.morgan_fp = MORGAN_FP
        # 5 kinetic + 13 spange + 122 drfp + 5 acs_pca + 1024 morgan = 1169 features
        self.feats_dim = 5 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1] + 1024

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            A_morgan = np.array([self.morgan_fp.get(s, np.zeros(1024)) for s in X["SOLVENT A NAME"]])
            B_morgan = np.array([self.morgan_fp.get(s, np.zeros(1024)) for s in X["SOLVENT B NAME"]])
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
                X_morgan = B_morgan * (1 - (1-pct)) + A_morgan * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
                X_morgan = A_morgan * (1 - pct) + B_morgan * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
            X_morgan = np.array([self.morgan_fp.get(s, np.zeros(1024)) for s in X["SOLVENT NAME"]])
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs, X_morgan])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'Combined feature dimension: {CombinedFeaturizer().feats_dim}')

In [ ]:
# GNNModel - A simple but effective model using Morgan fingerprints
# This is the SAME class that will be used in submission cells
class GNNModel(nn.Module):
    """GNN-inspired model using Morgan fingerprints as molecular representation.
    
    Uses a deep MLP with residual connections to process the combined features.
    The Morgan fingerprints capture structural information that tabular features miss.
    """
    def __init__(self, data='single'):
        super().__init__()
        self.data_type = data
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.scaler = StandardScaler()
        self.model = None
        
    def _build_model(self, input_dim):
        """Build the neural network with residual connections"""
        return nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(256, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(64, 3),
            nn.Sigmoid()
        )
    
    def train_model(self, X_train, y_train):
        # Featurize
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        # Data augmentation for mixtures (TTA during training)
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all = X_std
            y_all = y_vals
        
        # Scale features
        X_scaled = self.scaler.fit_transform(X_all)
        
        # Convert to tensors
        X_tensor = torch.tensor(X_scaled, dtype=torch.double).to(device)
        y_tensor = torch.tensor(y_all, dtype=torch.double).to(device)
        
        # Build model
        input_dim = X_tensor.shape[1]
        self.model = self._build_model(input_dim).double().to(device)
        
        # Training
        dataset = TensorDataset(X_tensor, y_tensor)
        loader = DataLoader(dataset, batch_size=32, shuffle=True)
        
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3, weight_decay=1e-5)
        criterion = nn.HuberLoss()
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=20, verbose=False
        )
        
        self.model.train()
        for epoch in range(300):
            epoch_loss = 0.0
            for inputs, targets in loader:
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                optimizer.step()
                epoch_loss += loss.item() * inputs.size(0)
            
            scheduler.step(epoch_loss / len(dataset))
    
    def predict(self, X):
        self.model.eval()
        
        # TTA for mixtures
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False)
            X_flip = self.featurizer.featurize(X, flip=True)
            
            X_std_scaled = self.scaler.transform(X_std)
            X_flip_scaled = self.scaler.transform(X_flip)
            
            X_std_tensor = torch.tensor(X_std_scaled, dtype=torch.double).to(device)
            X_flip_tensor = torch.tensor(X_flip_scaled, dtype=torch.double).to(device)
            
            with torch.no_grad():
                pred_std = self.model(X_std_tensor)
                pred_flip = self.model(X_flip_tensor)
                pred = (pred_std + pred_flip) / 2
        else:
            X_feat = self.featurizer.featurize(X)
            X_scaled = self.scaler.transform(X_feat)
            X_tensor = torch.tensor(X_scaled, dtype=torch.double).to(device)
            
            with torch.no_grad():
                pred = self.model(X_tensor)
        
        return pred.cpu()

print('GNNModel defined - will be used in both CV and submission cells')

In [ ]:
# Cross-validation to compute CV score
print("Computing CV score...")

# Single solvent CV
X_single, Y_single = load_data("single_solvent")
single_mses = []

for fold_idx, split in enumerate(generate_leave_one_out_splits(X_single, Y_single)):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = GNNModel(data='single')  # SAME CLASS AS SUBMISSION
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X).numpy()
    targets = test_Y.values
    
    mse = np.mean((predictions - targets) ** 2)
    single_mses.append(mse)
    
    if fold_idx % 6 == 0:
        print(f"  Fold {fold_idx}: MSE = {mse:.6f}")

single_mse = np.mean(single_mses)
print(f"\nSingle solvent MSE: {single_mse:.6f}")

In [ ]:
# Full data CV
X_full, Y_full = load_data("full")
full_mses = []

for fold_idx, split in enumerate(generate_leave_one_ramp_out_splits(X_full, Y_full)):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = GNNModel(data='full')  # SAME CLASS AS SUBMISSION
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X).numpy()
    targets = test_Y.values
    
    mse = np.mean((predictions - targets) ** 2)
    full_mses.append(mse)
    
    if fold_idx % 3 == 0:
        print(f"  Fold {fold_idx}: MSE = {mse:.6f}")

full_mse = np.mean(full_mses)
print(f"\nFull data MSE: {full_mse:.6f}")

In [ ]:
# Combined CV score
cv_score = (single_mse + full_mse) / 2
print(f"\n=== CV Results ===")
print(f"Single solvent MSE: {single_mse:.6f}")
print(f"Full data MSE: {full_mse:.6f}")
print(f"Combined CV score: {cv_score:.6f}")

# Save metrics
import json
metrics = {
    'cv_score': cv_score,
    'single_mse': single_mse,
    'full_mse': full_mse
}
with open('/home/code/experiments/115_proper_gnn_v2/metrics.json', 'w') as f:
    json.dump(metrics, f)

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = GNNModel(data='single')  # SAME CLASS AS CV
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = GNNModel(data='full')  # SAME CLASS AS CV
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

print(f"Submission saved with {len(submission)} rows")
print(submission.head())

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################